In [1]:
import numpy as np
import pandas as pd
import os
import re
import difflib
from collections import Counter
from metaphone import doublemetaphone

ROOT = '/home/thabib/database_records_linkage'

In [8]:
s1_s2_commonids = pd.read_csv(os.path.join(ROOT, 'data', 's1_s2_commonids.tsv'), sep='\t')
s1_cstr = pd.read_csv(os.path.join(ROOT,'data', 'source1_cstr.csv'))
s2_nond_cstr = pd.read_csv(os.path.join(ROOT, 'data', 'source2_nonanids_nod_cstr.csv'))

In [9]:
s1_cstr

,id,name,street_number,street_type,street_name,address_line2,postal_code,city
0,542012000015,societe des sucreries du marquenterre,NaN,rue,de la fontaine,NaN,80120,rue
1,542012000031,societe des sucreries du marquenterre,NaN,rte,nationale,NaN,62140,marconnelle
2,545011900028,le grand cafe et gassendi,NaN,NaN,saint christophe,route de marseille,4000,digne les bains
3,552017600016,hernas cartonnage,50.0,rue,pasteur,NaN,80210,feuquieres en vimeu
4,552017600032,hernas cartonnage,NaN,NaN,zone des hayettes,NaN,80210,chepy
...,...,...,...,...,...,...,...,...
1826164,90771119600010,bionormande,NaN,rue,de la grande flandre,lot 1 7 a,76270,neufchatel en bray
1826165,90771120400012,ahela bat,18.0,rue,saint clement,NaN,93200,saint denis
1826166,90771121200015,deco batiment,7.0,rue,auguste blanqui,NaN,93200,saint denis
1826167,90760906900014,chateaux merveilles,15.0,rue,de milan,NaN,75009,paris 9


In [12]:
unique_street_types = np.unique(s1_cstr['street_type'].dropna())

In [13]:
s2_nond_cstr

,Unnamed: 0,address,website,id,name
0,1,50 rte bage 01380 saint cyr sur menthon,NaN,84122632700010,sarl landrix freres
1,2,120 r menthon 01380 saint cyr sur menthon,NaN,79134582000025,marion kwasniewski
2,4,350 rte loeze 01380 saint cyr sur menthon,NaN,43416176600010,manigand stephane
3,5,41 r centre 01380 saint cyr sur menthon,http://www.infirmiere-frederique-thevenard.fr,48854644100012,frederique thevenard
4,6,116 chem cochatieres 01380 saint cyr sur menthon,NaN,41811188600028,garnier xavier
...,...,...,...,...,...
2271357,3530296,2 av chateau 83150 bandol,NaN,30130663500097,carlot jean francois
2271358,3530297,NaN,NaN,84978102600018,jacoby o
2271359,3530299,5 r jardins 83150 bandol,http://aquabike-en-mer.com,79518496900025,calvet richard
2271360,3530300,54 traverse laboureur 83150 bandol,http://dolcefarnientebandol.com,53925863200014,dolce far niente


# Testing Double Metaphone on name_x, name_y similarity matching

In [10]:
s1_s2_commonids.iloc[13]

Unnamed: 0                                                  13
id                                                565014800033
name_x                                 SCIC VALDURANCE HABITAT
street_number                                              2.0
street_type                                                RUE
street_name                            DU DOCTEUR SIMON PIETRI
address_line2                                              NaN
postal_code                                               4000
city                                           DIGNE-LES-BAINS
address          Bp 169 2 r Doct Piétri, 04000 DIGNE LES BAINS
website                                                    NaN
name_y                                 Valdurance Habitat SCIC
Name: 13, dtype: object

In [135]:
x_ = 'SCIC VALDURANCE HABITAT'.split(' ')
x_.sort()
x_ = ' '.join(x_)
print(doublemetaphone(x_))

y_ = 'Valdurance Habitat SCIC'.split(' ')
y_.sort()
y_ = ' '.join(y_)
print(doublemetaphone(y_))

('HPTTSKFLTRNS', '')
('HPTTSKFLTRNS', '')


In [137]:
y_

'Habitat SCIC Valdurance'

In [124]:
doublemetaphone('SCIC VALDURANCE HABITAT')

('SKFLTRNSPTT', '')

In [125]:
doublemetaphone('Valdurance Habitat SCIC')

('FLTRNSPTTSK', '')

In [45]:
s1_s2_commonids.iloc[770311]['name_x']

"SOCIETE D'EXERCICE LIBERALE A RESPONSABILITE LIMITEE DE CHIRURGIENS DENTISTES PATRICK SEBAG"

In [54]:
x = s1_s2_commonids.iloc[770311]['name_x'].split(' ')
x.sort()
x = ' '.join(x)
x

"A CHIRURGIENS D'EXERCICE DE DENTISTES LIBERALE LIMITEE PATRICK RESPONSABILITE SEBAG SOCIETE"

In [46]:
s1_s2_commonids.iloc[770311]['name_y']

'Chirurgiens Dentistes Patr'

In [58]:
y = s1_s2_commonids.iloc[770311]['name_y'].split(' ')
y.sort()
y = ' '.join(y)
y

'Chirurgiens Dentistes Patr'

In [59]:
doublemetaphone(x)

('AXRRJNSTTKSRSSTTNTSTSLPRLLMTPTRKRSPNSPLTSPKSST',
 'AKRRKNSTTKSRSSTTNTSTSLPRLLMTPTRKRSPNSPLTSPKSXT')

In [63]:
doublemetaphone(y)

('XRRJNSTNTSTSPTR', 'XRRKNSTNTSTSPTR')

In [145]:
print(get_overlap('THR', ''))

['']


In [132]:
''.join(get_overlap(doublemetaphone('Valdurance Habitat SCIC')[0], doublemetaphone('SCIC VALDURANCE HABITAT')[0]))

'FLTRNSPTT'

In [95]:
doublemetaphone('JANELI')

('JNL', 'ANL')

In [96]:
doublemetaphone('Intermarché Super')

('ANTRMRXSPR', 'ANTRMRKSPR')

# My Algorithm for Similarity MAtching for Strings

In [43]:
def get_overlap(s1, s2):
    # From: https://stackoverflow.com/questions/14128763/how-to-find-the-overlap-between-2-sequences-and-return-it
    s = difflib.SequenceMatcher(None, s1, s2)
    list_t = s.get_matching_blocks()
    #print(list_t)
    best_overlaps = []
    for i in range(len(list_t)):
        best_overlaps.append(s1[list_t[i][0]:list_t[i][0]+list_t[i][2]])
    return best_overlaps

def get_match_score(s1_name, s2_name):
    
    s1_ = s1_name.split(' ')
    s1_.sort()
    s1_ = ' '.join(s1_)
    s1_name_dmp = doublemetaphone(s1_)
    #print(s1_name_dmp)
    s2_ = s2_name.split(' ')
    s2_.sort()
    s2_ = ' '.join(s2_)
    s2_name_dmp = doublemetaphone(s2_)
    #print(s2_name_dmp)
    
    t1_t1_overlap = ''.join(get_overlap(s1_name_dmp[0], s2_name_dmp[0]))
    if s1_name_dmp[1] == '' and s2_name_dmp[1] == '':
        t2_t2_overlap = [1]
    else:
        t2_t2_overlap = ''.join(get_overlap(s1_name_dmp[1], s2_name_dmp[1]))
    t1_t2_overlap = ''.join(get_overlap(s1_name_dmp[0], s2_name_dmp[1]))
    t2_t1_overlap = ''.join(get_overlap(s1_name_dmp[1], s2_name_dmp[0]))
    
    t1_t1_score = len(t1_t1_overlap)/max(len(s1_name_dmp[0]), len(s2_name_dmp[0]), 1)
    t2_t2_score = len(t2_t2_overlap)/max(len(s1_name_dmp[1]), len(s2_name_dmp[1]), 1)
    t1_t2_score = len(t1_t2_overlap)/max(len(s1_name_dmp[0]), len(s2_name_dmp[1]), 1)
    t2_t1_score = len(t2_t1_overlap)/max(len(s1_name_dmp[1]), len(s2_name_dmp[0]), 1)
    
    match_score = t1_t1_score + 0.5*t2_t2_score + 0.25*(t1_t2_score + t2_t1_score)
    
    return match_score

In [45]:
get_match_score('av Gare, ANNOT','GARE DES CHEMINS DE FER DE PROVENCE ZONE DE LA GARE, ANNOT')

0.2884615384615385

In [46]:
get_match_score('Chirurgiens Dentistes Patr',"SOCIETE D'EXERCICE LIBERALE A RESPONSABILITE LIMITEE DE CHIRURGIENS DENTISTES PATRICK SEBAG")

0.625

In [48]:
get_match_score('Taher ALi Habib', 'HABIB Taher ALi')

1.5

In [50]:
get_match_score('pl Revelly, 04240 ANNOT', 'QUA QUARTIER DE LA BEITE,4240 ANNOT')

0.3352272727272727

In [52]:
get_match_score('rte 2 Vallées, 80100 ABBEVILLE', 'RTE DES DEUX VALLEES,80100 ABBEVILLE')

0.85

In [141]:
''.join(get_overlap(doublemetaphone('Taher')[0], doublemetaphone('Taher')[0]))

'THR'

In [142]:
''.join(get_overlap(doublemetaphone('Taher')[1], doublemetaphone('Taher')[1]))

''

In [143]:
''.join(get_overlap(doublemetaphone('Taher')[0], doublemetaphone('Taher')[1]))

''

In [144]:
''.join(get_overlap(doublemetaphone('Taher')[1], doublemetaphone('Taher')[0]))

''

In [ ]:
from thefuzz import fuzz

In [38]:
fuzz.token_sort_ratio('av Gare, ANNOT','GARE DES CHEMINS DE FER DE PROVENCE ZONE DE LA GARE, ANNOT')

31

In [47]:
fuzz.token_sort_ratio('Chirurgiens Dentistes Patr',"SOCIETE D'EXERCICE LIBERALE A RESPONSABILITE LIMITEE DE CHIRURGIENS DENTISTES PATRICK SEBAG")

44

In [49]:
fuzz.token_sort_ratio('Taher ALi Habib', 'HABIB Taher ALi')

100

In [51]:
fuzz.token_sort_ratio('pl Revelly, 04240 ANNOT', 'QUA QUARTIER DE LA BEITE,4240 ANNOT')

53

In [53]:
fuzz.token_sort_ratio('rte 2 Vallées, 80100 ABBEVILLE', 'RTE DES DEUX VALLEES,80100 ABBEVILLE')

81

### Conclusions

1. _String Fuzzy matcher from the library __'thefuzz'__ is performing quite better than my own function_
2. It can match both names as well as addresses equally reliably

See here: https://github.com/seatgeek/thefuzz